In [3]:
from bson.json_util import dumps, loads
from bson.objectid import ObjectId
from pymongo import MongoClient
from pprint import pprint
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time
from collections import Counter

client = MongoClient('localhost:27017')
db = client.ContactDB

/Users/davis/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [124]:
def round_time(dt=None, delta=timedelta(minutes=1), to='average'):
    """
    Round a datetime object to a multiple of a timedelta
    dt : datetime.datetime object, default now.
    dateDelta : timedelta object, we round to a multiple of this, default 1 minute.
    from:  http://stackoverflow.com/questions/3463930/how-to-round-the-minute-of-a-datetime-object-python
    """
    minDt = datetime.now().replace(month=1, day=1, hour=0, minute=0, second=0, microsecond=0)  
    
    round_to = delta.total_seconds()
    if dt is None:
        dt = datetime.now()
    seconds = (dt - minDt).seconds

    if seconds % round_to == 0 and dt.microsecond == 0:
        rounding = (seconds + round_to / 2) // round_to * round_to
    else:
        if to == 'ceil':
            # // is a floor division, not a comment on following line (like in javascript):
            rounding = (seconds + dt.microsecond/1000000 + round_to) // round_to * round_to
        elif to == 'floor':
            rounding = seconds // round_to * round_to
        else:
            rounding = (seconds + round_to / 2) // round_to * round_to

    return dt + timedelta(0, rounding - seconds, - dt.microsecond)

In [5]:
def generate_next_summary(station_id, summary_period, current_dateTime=None):
    ## Generate the next summary of a days events
    
    if current_dateTime is None:
        current_dateTime = datetime.now()
        
    # Get latest summary dateTime
    summary_cursor = list(db.daily_summaries.find({}).sort([("_id",-1)]).limit(1))
    if len(summary_cursor) == 0:
        # No current summary entries, use first event as latest datetime
        all_events = list(db.touch_events.find({}).sort([("_id",-1)]))
        latest_summary_dateTime = all_events[-1]['move_datetime']
    else:
        latest_summary_dateTime = summary_cursor[0]['datetime']
        
    # floor the most recent datetime to summary perioud
    floored_latest_summary_dateTime = round_time(latest_summary_dateTime, delta=summary_period, to='floor')
    # Cutoff for next summary is the previous floored summary datetime + the delta
    next_summary_dateTime = floored_latest_summary_dateTime + summary_period 
    
        
    if current_dateTime > next_summary_dateTime:  # If last summary was more then the summary period ago, make a new summary
        
        # Convert from datetime back into timestamp
        next_summary_dateTime = datetime.timestamp(next_summary_dateTime) * 1000

        ## Time to generate new summary
        # Gets all events in the 6hr period following the most recent summary
        next_touch_events = db.touch_events.find({
            '$and': [
                {'load_station_id': station_id},
                {'move_datetime': {'$gt': latest_summary_dateTime}},
                {'move_datetime': {'$lte': next_summary_dateTime}}
            ]
        })
        
        next_report_events = db.report_events.find({
            '$and': [
                {'load_station_id': station_id},
                {'datetime': {'$gt': latest_summary_dateTime}},
                {'datetime': {'$lte': next_summary_dateTime}}
            ]
        })

        return summarize_events(next_touch_events, next_report_events, next_summary_dateTime)

    else:   # Otherwise there is no new summary, return None
        print('nothing to summarize')
        return None

In [6]:
def summarize_events(touch_events, report_events, summary_date):
    
    return True

In [210]:
def bin_quantities_to_line_data(df, bins):
    # Pass this a df and bins and it will output line data with the x being the bin and y being the summation of quantity for events within the bin window
    cut_data = pd.cut(df['move_datetime'], bins)
    grouped_data = list(df.groupby(cut_data)['quantity'].sum())
    
    return list(map(lambda x, y: {'x': x, 'y': y}, bins, grouped_data))

def bin_wip_to_lines_data(df, bins, product_groups):
    cut_data = pd.cut(df['move_datetime'], bins)
    grouped_data = list(df.groupby(cut_data)['current_wip'])

    wip_data_dict = {}
    for pg in product_groups:
        wip_data_dict[pg['_id']] = {
            'id': pg['name'],
            'data': []
        }
        
    for bin_data in grouped_data:
        
        values_df = pd.DataFrame(bin_data[1].values.tolist())
        max_wip = {key: values_df[key].max() for key in values_df.columns}
        
        for pg in product_groups:
            if pg['_id'] in max_wip:
                wip_data_dict[pg['_id']]['data'].append({'x': bin_data[0].right, 'y': max_wip[pg['_id']]})
            else:
                wip_data_dict[pg['_id']]['data'].append({'x': bin_data[0].right, 'y': 0})
            
    return wip_data_dict.values()

def product_group_pie(df, product_groups):
    grouped_df = list(df.groupby('product_group_id'))
    
    pg_pie = []
    for pg_id, pg_df in grouped_df:
        pg_pie.append({
            'id': pg_id,
            'label': product_groups[pg_id]['name'],
            'value': pg_df['quantity'].sum()
        })
        
    return pg_pie

def bin_reports_to_bar_data(df, bins, colors):
    # Bin data
    cut_data = pd.cut(df['datetime'], bins)
    binned_data = list(df.groupby(cut_data))

    bar_data = []
    for data in binned_data:
        bar = {
            "x": data[0].right # X value is the timestep to the right of the bin
        }
        # Organize the reports by name, and set the bar to the count of instances of that report
        value_counts = data[1]['report_name'].value_counts().to_dict()
        for key, value in value_counts.items():
            bar[key] = value
        
        # Populate the bar with the colors that we mapped earlier
        for report_name, color in colors.items():
            bar[report_name + 'Color'] = color
            
        bar_data.append(bar)

    return bar_data

def reports_pie(df, colors):
    report_counts = df['report_name'].value_counts().to_dict()
    
    report_pie = []
    for report_name, count in report_counts.items():
        report_pie.append({
            "id": report_name,
            "label": report_name, 
            "value": count,
            "color": colors[report_name]
        })
        
    return report_pie

def generate_statistics_from_events(station_id, touch_events, report_events):
    touches_df = pd.DataFrame(touch_events)    
    te_count = len(touch_events)
        
    station = db.stations.find_one({'_id': station_id})
    
    stats = {}
    
    if te_count == 0:
        product_group_ids = []
    else:
        product_group_ids = touches_df['product_group_id'].unique()
    product_groups = {}
    for pg_id in product_group_ids:
        product_groups[pg_id] = db.lot_templates.find_one({'_id': pg_id})

    # TODO: Implement this and delete from this function
    station['desired_cycle_times'] = {}
    station['theo_min_cycle_times'] = {}
    for pg_id in product_group_ids:
        station['desired_cycle_times'][pg_id] = 1
        station['theo_min_cycle_times'][pg_id] = 0.8
    # ===================================================
    
    stats['efficiency'] = []
    stats['oee'] = []
    stats['cycle_time'] = {}
    
    found_pg_ids = []
    for touch_event in touch_events[::-1]: # Reverse order because we want to take the stats at the end of the day
        pg_id = touch_event['product_group_id']
                
        # First time weve found this product group, this is the latest data we have
        if pg_id not in found_pg_ids:
            found_pg_ids.append(pg_id)
            
            # --- Efficieny
            stats['efficiency'].append({
                'id': product_groups[pg_id]['name'],
                'data': [{
                    'x': 'Efficiency',
                    'y': station['desired_cycle_times'][pg_id] / touch_event['pgs_cycle_time']
                }]
            })
            
            # --- OEE
            stats['oee'].append({
                'id': product_groups[pg_id]['name'],
                'data': [{
                    'x': 'OEE',
                    'y': station['theo_min_cycle_times'][pg_id] / touch_event['pgs_cycle_time']
                }]
            })
            
            # --- Cycle Time
            stats['cycle_time'][pg_id] = {
                'current': touch_event['pgs_cycle_time'],
                'line_data': [{
                    'id': product_groups[pg_id]['name'],
                    'data': [{
                        'x': touch_event['move_datetime'],
                        'y': touch_event['pgs_cycle_time']
                    }]
                }]
            }
            
        else:
            # --- Cycle Time
            # There is already data for this product group, insert it at beginnning (since we are going through events in reverse order)
            stats['cycle_time'][pg_id]['line_data'][0]['data'].insert(0, {
                'x': touch_event['move_datetime'],
                'y': touch_event['pgs_cycle_time']
            })
            
    # === Generate bins
    if te_count == 0:
        bins = []
    else:
        start_hour = round_time(touch_events[0]['move_datetime'], timedelta(hours=1), 'floor')
        end_hour = round_time(touch_events[-1]['move_datetime'], timedelta(hours=1), 'ceil')
        print(start_hour, end_hour)
        delta_hours = (end_hour - start_hour).seconds // 3600
        
        bins = [start_hour]
        for delta_hours in range(delta_hours):
            bins.append(start_hour + timedelta(hours=delta_hours+1))
        

    # === LINE GRAPHS
    stats['throughput'] = []
    stats['wip'] = []
    stats['product_group_pie'] = []
    stats['reports'] = []
    stats['reports_pie'] = []
    
    if te_count > 0:
        # --- Throughput
        stats['throughput'].append({
            'id': 'Total',
            'data': bin_quantities_to_line_data(touches_df, bins)
        })
        for pg_id in product_group_ids:
            stats['throughput'].append({
                'id': product_groups[pg_id]['name'],
                'data': bin_quantities_to_line_data(touches_df[touches_df['product_group_id'] == pg_id], bins)
            })
            
        # --- WIP
        stats['wip'] = bin_wip_to_lines_data(touches_df, bins, product_groups.values())
        
        # --- Product Group Pie
        stats['product_group_pie'] = product_group_pie(touches_df, product_groups)
        
    if len(report_events) > 0:
        # === Reports
        # Instead of querying the dashboards every time, fill in the dataframe with report name and report color
        dashboard = db.dashboards.find_one({'station': station_id})
        report_buttons_normalized = {x['_id']: x for x in dashboard['report_buttons']}

        reports_colors = {} # Also store a dictionary of all colors, with the key being the report name
        for i, re in enumerate(report_events):
            rb_id = re['report_button_id']
            if rb_id in report_buttons_normalized:
                report_name = report_buttons_normalized[rb_id]['label']
                report_events[i]['report_name'] = report_name
                reports_colors[report_name] = report_buttons_normalized[rb_id]['color']

        # Convert to dataframe
        reports_df = pd.DataFrame(report_events)

        # --- Reports Bar
        stats['reports'] = bin_reports_to_bar_data(reports_df, bins, reports_colors)
        
        # --- Reports Pie
        stats['reports_pie'] = reports_pie(reports_df, reports_colors)
            
    return stats['product_group_pie']

In [211]:
station_id = '35dc9b69-3d93-4a83-8a25-99432edc43b0'
day_start = round_time(datetime.today(), timedelta(days=1), 'floor')
print(day_start)

tes = list(db.touch_events.find({'$and': [{'load_station_id': station_id}, {'move_datetime': {'$gte': day_start}}]}).sort([("move_datetime",1)]))
res = list(db.report_events.find({'$and': [{'station_id': station_id}, {'datetime': {'$gte': day_start}}]}).sort([("move_datetime",1)]))

# TODO: Implement this and remove from here
for i, te in enumerate(tes):
    tes[i]['pgs_cycle_time'] = calculate_pg_station_cycle_time(te['load_station_id'], te['product_group_id'])
    if tes[i]['pgs_cycle_time'] == 0:
        tes[i]['pgs_cycle_time'] = 10

pprint(generate_statistics_from_events(station_id, tes, res))

2021-11-05 00:00:00
2021-11-05 13:00:00 2021-11-05 14:00:00
[{'id': '6166150652325faeb6d12b7d', 'label': 'Snowboard', 'value': 10},
 {'id': '6172ebb010e2161e81d01f2e', 'label': 'Basic', 'value': 10},
 {'id': '6180250995773c687755558b', 'label': 'Davis', 'value': 45}]


In [ ]:
df = pd.DataFrame(res)

# == = = = = = = = = == = 



In [9]:
def generate_statistics_from_summaries(summaries):
    return

In [10]:
{
    # Efficiency
    'efficiency': {
        '[product_group_id]': 100
    },
    'overall_efficiency': 100,
    
    # OEE
    'oee': {
        '[product_group_id]': 100
    },
    'overall_oee': 100,
    
    # Cycle Time
    'cycle_time': {
        '[product_group_id]': {
            'line_data': [{'x': None, 'y': None}],
            'current_cycle_time': "00:00:00"
        }
    },
    
    # Throughput
    'throughput': {
        '_total': [
            {'x': None},
            {'y': None}
        ],
        '[product_group_id]' : [
            {'x': None},
            {'y': None}
        ]
    },
    
    # WIP (stacked histogram)
    'wip': [
        {
            'x': None,
            '[product_group_name]': None
        }
    ],
    
    # Station Reports
    'station_reports': [
        {
            'x': None,
            '[product_group_name]': None
        }
    ],
    
    # Pie Charts
    'station_reports_pie': {
        '[report_name]': 100
    },
    'product_group_pie': {
        '[product_group_name]': 100
    },
    'operator_pie': {
        '[operator]': 100
    },
    'output_station': {
        '[station_name]': 100
    },
    
    # Machine Utilization
    'machine_utilization': [
        {
            'Working': 100,
            'WorkingColor': 'green'
        },
        {
            'Idle': 100,
            'IdleColor': 'green'
        }
    ],
    
    # Value Creating Time
    'value_creating_time': [
        {
            'Working': 100,
            'WorkingColor': 'green'
        },
        {
            'Idle': 100,
            'IdleColor': 'green'
        }
    ],
    
    # NOTE: This needs to pe paginated somehow
    'touch_events': [],
    'report_events': []
    
}

{'efficiency': {'[product_group_id]': 100},
 'overall_efficiency': 100,
 'oee': {'[product_group_id]': 100},
 'overall_oee': 100,
 'cycle_time': {'[product_group_id]': {'line_data': [{'x': None, 'y': None}],
   'current_cycle_time': '00:00:00'}},
 'throughput': {'_total': [{'x': None}, {'y': None}],
  '[product_group_id]': [{'x': None}, {'y': None}]},
 'wip': [{'x': None, '[product_group_name]': None}],
 'station_reports': [{'x': None, '[product_group_name]': None}],
 'station_reports_pie': {'[report_name]': 100},
 'product_group_pie': {'[product_group_name]': 100},
 'operator_pie': {'[operator]': 100},
 'output_station': {'[station_name]': 100},
 'machine_utilization': [{'Working': 100, 'WorkingColor': 'green'},
  {'Idle': 100, 'IdleColor': 'green'}],
 'value_creating_time': [{'Working': 100, 'WorkingColor': 'green'},
  {'Idle': 100, 'IdleColor': 'green'}],
 'touch_events': [],
 'report_events': []}

In [11]:
def calc_daily_working_seconds(shift_details, start_time, stop_time):
    
    begin_shift = time.strptime(shift_details['startOfShift'],'%H:%M')
    begin_shift_secs = timedelta(hours=begin_shift.tm_hour,minutes=begin_shift.tm_min,seconds=begin_shift.tm_sec).total_seconds()

    # Cant start before the shift starts
    if start_time is not None:
        start_time_secs = timedelta(hours=start_time.hour,minutes=start_time.minute,seconds=start_time.second).total_seconds()
        start_time_secs = max(start_time_secs, begin_shift_secs)
    else: 
        start_time_secs = begin_shift_secs
    
    end_of_shift = time.strptime(shift_details['endOfShift'],'%H:%M')
    end_of_shift_secs = timedelta(hours=end_of_shift.tm_hour,minutes=end_of_shift.tm_min,seconds=end_of_shift.tm_sec).total_seconds()
    
    if stop_time is not None:
        stop_time_secs = timedelta(hours=stop_time.hour,minutes=stop_time.minute,seconds=stop_time.second).total_seconds()
        end_time_secs = min(stop_time_secs, end_of_shift_secs)
    else:
        end_time_secs = end_of_shift_secs
    
    # Total working seconds between the start and stop not including breaks
    working_seconds = max(0, (end_time_secs - start_time_secs))
    
    # Need to calculate the working time using shift details 
    breaks = shift_details['breaks'].values()
    # For each break, calculate the length of the break in seconds and ad to total break time
    for br in breaks:
        if(br['enabled']):
            br_start_time = time.strptime(br['startOfBreak'],'%H:%M')
            br_start_secs = timedelta(hours=br_start_time.tm_hour,minutes=br_start_time.tm_min,seconds=br_start_time.tm_sec).total_seconds()
            br_start_secs = min(start_time_secs, max(br_start_secs, stop_time_secs))
            
            br_end_time = time.strptime(br['endOfBreak'],'%H:%M')
            br_end_secs = timedelta(hours=br_end_time.tm_hour,minutes=br_end_time.tm_min,seconds=br_end_time.tm_sec).total_seconds()
            br_end_secs = max(start_time_secs, min(br_end_secs, stop_time_secs))
            
            working_seconds -= (br_end_secs - br_start_secs)
            
    return working_seconds
    

def calculate_pg_station_cycle_time(station_id, product_group_id, new_event=None):
    events = list(db.touch_events.find({'$and': [{'load_station_id':station_id}, {'product_group_id': product_group_id}]}).sort('move_datetime').limit(30))
    if new_event is not None:
        events.append(new_event)
    settings = db.settings.find_one()
    
    # Fetch data
    df = pd.DataFrame(events)
    
    if len(df) > 1:
    
        
        shift_details = settings['shiftDetails']
        begin_shift = time.strptime(shift_details['startOfShift'],'%H:%M')
        begin_shift_secs = timedelta(hours=begin_shift.tm_hour,minutes=begin_shift.tm_min,seconds=begin_shift.tm_sec).total_seconds()
        end_of_shift = time.strptime(shift_details['endOfShift'],'%H:%M')
        end_of_shift_secs = timedelta(hours=end_of_shift.tm_hour,minutes=end_of_shift.tm_min,seconds=end_of_shift.tm_sec).total_seconds()
        
        # Total quantity of parts moved
        total_quantity = 0
        for i, event in df.iterrows():
            # Only count moves that were within the shift
            event_stop_time = event['move_datetime']
            event_stop_time_secs = timedelta(hours=event_stop_time.hour,minutes=event_stop_time.minute,seconds=event_stop_time.second).total_seconds()
            if event_stop_time_secs > begin_shift_secs and event_stop_time_secs < end_of_shift_secs:
                total_quantity += event['quantity']
                                
        # If quantity is 0, nothing we can do
        if total_quantity == 0:
            return 0
        
        first_event_time = df.iloc[0]['move_datetime']
        last_event_time = df.iloc[-1]['move_datetime']
                
        
        if first_event_time.date() == last_event_time.date():
            total_working_seconds = calc_daily_working_seconds(shift_details, first_event_time.time(), last_event_time.time())
        else:
            first_day_working_seconds = calc_daily_working_seconds(shift_details, first_event_time.time(), None)
            last_day_working_seconds = calc_daily_working_seconds(shift_details, None, last_event_time.time())
            
            daily_working_seconds = calc_daily_working_seconds(shift_details, None, None)
            
            # Find unique dates to be used later
            unique_dates = []
            for datetime in df['move_datetime']:
                rounded_date = datetime.date()
                if rounded_date not in unique_dates: unique_dates.append(rounded_date)
                
            total_working_seconds = first_day_working_seconds + daily_working_seconds*(len(unique_dates)-2) + last_day_working_seconds
            
        cycle_time = total_working_seconds / total_quantity
        return cycle_time   
                     
    else:
        return 0
        

In [12]:
def generate_summaries(station_id, current_dateTime=None):
     # summarize all 10min summaries to catch up to current date
    summary = generate_next_summary(station_id, timedelta(days=1), current_dateTime)
    while summary is not None:
        # db.historicals_day.insert_one(summary)
        summary = generate_next_summary(station_id, timedelta(days=1), current_dateTime)